In [1]:
# 自动加载修改过的文件
%load_ext autoreload
%autoreload 2
import os
import sys
import warnings
sys.path.append('../')  # |返回notebook的上一级目录
warnings.filterwarnings("ignore")

In [5]:
import tensorflow as tf
from utils.config_gpu import config_gpu
config_gpu()

1 Physical GPUs, 1 Logical GPUs


In [3]:
from utils.config import *
from utils.saveLoader import load_text, get_text, load_dataset

In [36]:
@tf.function
def f(a):
    print("The function is running in Python")
    b = [tf.constant(3., dtype=tf.float32)]
    b += a
    tf.print(b)

In [43]:
a = [tf.constant(11., dtype=tf.float32), tf.constant(7., dtype=tf.float32)]
f(a)

[3, 11, 7]


In [15]:
a = tf.TensorArray(dtype=tf.float32, dynamic_size=True)

ValueError: Size must be declared for TensorArrays when eager execution is enabled.

In [1]:
a = [14712,480,1111,308,288,480,23,90,121,308,288,480,480,543,308,10,1111,554,27,152,5,34,11,665,1,68,539,1805,258,459,1111,75,722,1111,14713,491,744,385,263,20,431,14713,4939,288,141,103,457,659,5349,33,64,69,94,1071,14,385,2198,14714,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715,14715]